In [23]:
#resolve Dependencies
import pandas as pd
import os
import pymongo
from pymongo import MongoClient

In [24]:
file1 = ''
file2 = ''
file3 = ''
file4 = ''
file5 = ''



In [25]:
free_reduced_src_path = os.path.join('..','static', 'data', 'clean', 'free_reduced_lunch_ny_county.csv')
free_reduced_src_path








'..\\static\\data\\clean\\free_reduced_lunch_ny_county.csv'

In [26]:
free_reduced_df = pd.read_csv(free_reduced_src_path)

In [27]:
free_reduced_df.head()

,id,region,population,number on program,percent of the county,percent of state
0,0500000US36NYS,New York State,19453561,"1,441,350",NaN,7.41
1,0500000US36NYC,New York City,8336817,"775,759",NaN,3.99
2,0500000US36NYR,Rest of State,11116744,"665,591",NaN,3.42
3,0500000US36047,Kings County,2559903,"238,173",9.0,1.22
4,0500000US36081,Queens County,2253858,"204,916",9.0,1.05


In [28]:
# connect to mongodb
client = MongoClient('mongodb://localhost:27017')

In [29]:
#set db connection
db = client['nycares_db']

In [30]:
#set connection to collection
frlp_collection = db['free_and_reduced_lunch']

In [31]:
#create a dictionary to add mongo records
data_as_dict = free_reduced_df.to_dict("records")

In [32]:
frlp_collection.insert_one({"index": "region", "data":data_as_dict})